<a href="https://colab.research.google.com/github/KettoMisaell/UFC_PREDICTION_XGBOOST/blob/main/UFC_XGBOOST_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PREDICCIONES DE LOS GANADORES DE COMBATES DE UFC CON XGBOOST**

Se utilizará el módelo XGBoost para decidir el ganador de cada combate de UFC registrado en el Data Set de Kaggle https://www.kaggle.com/datasets/mdabbert/ultimate-ufc-dataset, con 119 características diferentes por cada combate, estadísticas, de antes y después del combate, por supuesto para predecir el ganador solamente se ocuparán características de antes del combate.

Anteriormente se intentó hacer predicciones con un módelo de regresión logística alcanzando un aproximado de 65% de exito en las predicciones tras varias pruebas cambiando las variables predictoras (https://github.com/KettoMisaell/UFC_logistic_prediction/blob/main/UFC_bouts_regresion_logistica.ipynb)

Con este proyecto se busca probar la efectividad de XGBoost un algoritmo ensamblado con boosting que busca reducir las bias y variance creando un modelo base e iterando para corregir errores, creando modelos complementarios que corrijan los errores de los anterioes.

Este modelo suele presentar buenos resultados para data estructurada la cuál es nuestro tipo de data, y un reflejo de esto es su dominio en las competiciones de Kaggle.

# Carga y procesamiento

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
df = pd.read_csv('/content/ufc-master.csv')

Llenar valores nulos ates de trabajar con el Data Frame

In [3]:
df.fillna(0, inplace = True )

Columnas categoricas y numericas

In [ ]:
columnas_categoricas = [x for x in df.columns if df[x].dtype not in ['int64', 'float64']]

In [ ]:
len(columnas_categoricas)

15

In [ ]:
len(df.columns)

119

In [4]:
columnas_numericas = [x for x in df.columns if df[x].dtype in ['int64', 'float64']]

Sólo para comprobar, ninguna de las 15 columnas númericas parece poder aportar algo significativo a nuestro modelo así que no es necesario convertirlas.

Eliminamos los datos que no parezcan relevantes

In [5]:
elementos_a_eliminar = ['R_odds', 'B_odds','B_draw', 'R_draw', 'no_of_rounds' , 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'empty_arena', 'constant_1', 'B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank', 'finish_round', 'total_fight_time_secs']

# Crear una nueva lista sin los elementos a eliminar
nueva_lista = [x for x in columnas_numericas if x not in elementos_a_eliminar]

In [ ]:
len(nueva_lista)

63

# Separación del target y caracteristicas

In [6]:
df['Winner'] = np.where(df['Winner'] == 'Red', 1, 0)

In [7]:
X = df[nueva_lista]
y = df['Winner']

# Normalización

Cómo sugerencia de mi tutor probé con una normalización de mis datos con un rango de 0.1 a 0.9 utilizando MinMaxScaler de la libreria sklearn, sin embargo para mis resultados finales, parecía haber una mejora significativa en mis resultados utilizando StandardScaler de la misma libreria, de cualquier forma dejaré ambos metodos por si a alguien le resulta mejor uno u otro.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

columnas_a_normalizar = X.columns

scaler = MinMaxScaler(feature_range=(0.1, 0.9))

# Aplicar la normalización a las columnas seleccionadas
X[columnas_a_normalizar] = scaler.fit_transform(X[columnas_a_normalizar])


In [8]:
from sklearn.preprocessing import StandardScaler

# Crea el objeto de la clase StandardScaler
scaler = StandardScaler()

# Normaliza cada columna del dataframe por separado
for column in X.columns:
    X[column] = scaler.fit_transform(X[column].values.reshape(-1, 1))

# Division de la data de entrenamiento y test

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import statsmodels.api as sm

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y.values.reshape(-1,1),
                                        train_size   = 0.7,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

En el caso de la división probé con distintos tamaños de entrenamiento y con el que obtuve mejores resultados es 70% de data de entrenamiento y 30% de test.

In [ ]:
y_test.shape, X_train.shape

((1469, 1), (3427, 63))

Se realiza una segunda división sobre los datos de test para apartar datos de validación diferentes a con los que se entrena el módelo y prueba.

In [10]:
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5,random_state= 1234, shuffle      = True)

In [ ]:
y_test.shape, X_train.shape

((734, 1), (3427, 63))

# Creación del modelo

In [11]:
import xgboost

In [12]:
xgb = xgboost.XGBClassifier()

# HIPERPARAMETROS DEL MODELO

Se construye una red de hiperparámetros sobre los que se buscarán las mejores combinaciones para nuestro módelo, cabe destacar que existe un mayor número de hiperparámetros a tomar en cuenta para el modelo XGBOOST, cómo primera aproximación sugiero empezar con un número reducido de estos pues en mi experiencia al crecer el número de hiperparámetros no sólo tomaba un mayor tiempo de entrenamiento, sino que este tiempo de entrenamiento terminaba siendo en vano pues al tener que tomar en cuenta todos los hiperparámetros añadidos muchas veces la precisión de modelo empeoraba.

In [13]:
parameters = {'nthreads': [1],
              'objective': ['binary:logistic'],
              'learning_rate': [.3, .03, .003, .0003],
              'nestimators': [100, 150, 200],
              'max_depth': [3, 6, 8, 10],
                }

In [14]:
fit_params = {'early_stopping_rounds': 5,
              'eval_metric': 'logloss',
              'eval_set': [(X_test, y_test)]}

# BUSQUEDA DE HIPERPARAMETROS CON VALIDACION CRUZADA

In [15]:
from sklearn.model_selection import GridSearchCV

Se asigna la validacion cruzada a una variable, se le pasan como parametros el modelo, los hiperparametros a variar, los pliegues y como medirá la eficiencia

In [16]:
cfl = GridSearchCV(xgb, parameters, cv = 3, scoring = 'accuracy' )

In [17]:
cfl.fit(X_train, y_train, **fit_params)

[0]	validation_0-logloss:0.64325
[1]	validation_0-logloss:0.62736
[2]	validation_0-logloss:0.62009
[3]	validation_0-logloss:0.61472
[4]	validation_0-logloss:0.61316
[5]	validation_0-logloss:0.61115
[6]	validation_0-logloss:0.61367
[7]	validation_0-logloss:0.61321
[8]	validation_0-logloss:0.61388
[9]	validation_0-logloss:0.61519


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [23:54:39] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "nestimators", "nthreads" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboo

[0]	validation_0-logloss:0.63912
[1]	validation_0-logloss:0.62507
[2]	validation_0-logloss:0.61943
[3]	validation_0-logloss:0.61766
[4]	validation_0-logloss:0.61486
[5]	validation_0-logloss:0.61294
[6]	validation_0-logloss:0.61443
[7]	validation_0-logloss:0.61465
[8]	validation_0-logloss:0.61637
[9]	validation_0-logloss:0.61594
[10]	validation_0-logloss:0.61782
[0]	validation_0-logloss:0.63883
[1]	validation_0-logloss:0.62491
[2]	validation_0-logloss:0.61551
[3]	validation_0-logloss:0.61085
[4]	validation_0-logloss:0.60836
[5]	validation_0-logloss:0.60585
[6]	validation_0-logloss:0.60503
[7]	validation_0-logloss:0.60432
[8]	validation_0-logloss:0.60531


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [23:54:39] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "nestimators", "nthreads" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [23:54:39] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "nestimators", "nthreads" } are not used.

  warnings.warn(smsg, UserWarning)


[9]	validation_0-logloss:0.60654
[10]	validation_0-logloss:0.60667
[11]	validation_0-logloss:0.60742
[12]	validation_0-logloss:0.60712
[0]	validation_0-logloss:0.64325
[1]	validation_0-logloss:0.62736
[2]	validation_0-logloss:0.62009
[3]	validation_0-logloss:0.61472
[4]	validation_0-logloss:0.61316
[5]	validation_0-logloss:0.61115
[6]	validation_0-logloss:0.61367
[7]	validation_0-logloss:0.61321
[8]	validation_0-logloss:0.61388
[9]	validation_0-logloss:0.61519
[10]	validation_0-logloss:0.61554


...

[5]	validation_0-logloss:0.67223
[6]	validation_0-logloss:0.67219
[7]	validation_0-logloss:0.67215
[8]	validation_0-logloss:0.67210
[9]	validation_0-logloss:0.67206
[10]	validation_0-logloss:0.67202
[11]	validation_0-logloss:0.67197
[12]	validation_0-logloss:0.67193
[13]	validation_0-logloss:0.67189
[14]	validation_0-logloss:0.67184
[15]	validation_0-logloss:0.67180
[16]	validation_0-logloss:0.67176
[17]	validation_0-logloss:0.67172
[18]	validation_0-logloss:0.67167
[19]	validation_0-logloss:0.67163
[20]	validation_0-logloss:0.67159
[21]	validation_0-logloss:0.67155
[22]	validation_0-logloss:0.67150
[23]	validation_0-logloss:0.67146
[24]	validation_0-logloss:0.67142
[25]	validation_0-logloss:0.67138
[26]	validation_0-logloss:0.67134
[27]	validation_0-logloss:0.67129
[28]	validation_0-logloss:0.67125
[29]	validation_0-logloss:0.67121
[30]	validation_0-logloss:0.67117
[31]	validation_0-logloss:0.67113
[32]	validation_0-logloss:0.67108
[33]	validation_0-logloss:0.67104
[34]	validation_0-l

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [23:58:49] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "nestimators", "nthreads" } are not used.

  warnings.warn(smsg, UserWarning)


[1]	validation_0-logloss:0.67241
[2]	validation_0-logloss:0.67238
[3]	validation_0-logloss:0.67234
[4]	validation_0-logloss:0.67230
[5]	validation_0-logloss:0.67226
[6]	validation_0-logloss:0.67222
[7]	validation_0-logloss:0.67218
[8]	validation_0-logloss:0.67214
[9]	validation_0-logloss:0.67210
[10]	validation_0-logloss:0.67207
[11]	validation_0-logloss:0.67203
[12]	validation_0-logloss:0.67199
[13]	validation_0-logloss:0.67195
[14]	validation_0-logloss:0.67191
[15]	validation_0-logloss:0.67187
[16]	validation_0-logloss:0.67184
[17]	validation_0-logloss:0.67180
[18]	validation_0-logloss:0.67176
[19]	validation_0-logloss:0.67172
[20]	validation_0-logloss:0.67168
[21]	validation_0-logloss:0.67164
[22]	validation_0-logloss:0.67161
[23]	validation_0-logloss:0.67157
[24]	validation_0-logloss:0.67153
[25]	validation_0-logloss:0.67149
[26]	validation_0-logloss:0.67145
[27]	validation_0-logloss:0.67142
[28]	validation_0-logloss:0.67138
[29]	validation_0-logloss:0.67134
[30]	validation_0-loglo

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [23:58:53] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "nestimators", "nthreads" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'learning_rate': [0.3, 0.03, 0.003, 0.0003],
                         'max_depth': [3, 6, 8, 10],
                         'nestimators': [100, 150, 200], 'nthreads': [1],
                         'objective': ['binary:logistic']},
             scoring='accuracy')

El mejor modelo con la asignación de hiperparametros y parametros correspondiente despues del entrenamiento y validación cruzada

In [18]:
cfl.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              nestimators=100, nthreads=1, ...)

El mayor puntaje de accuracy durante el entrenamiento

In [19]:
cfl.best_score_

0.6396244763041513

# Evaluación con el Data Frame completo

In [20]:
from sklearn.metrics import accuracy_score

Se asigna el mejor modelo a una nueva variable

In [21]:
best_xgb = cfl.best_estimator_

Se realizan y guardan las predicciones

In [22]:
y_preds = best_xgb.predict(X_valid)

In [23]:
y_valid = y_valid.reshape(-1)

In [ ]:
y_valid.shape

(735,)

Se realiza una tabla de comparación entre los datos reales y los predichos

In [24]:
comp = pd.DataFrame({'real': y_valid, 'predicho': y_preds})

In [25]:
comp

,real,predicho
0,1,0
1,0,0
2,0,0
3,1,0
4,1,0
...,...,...
730,1,0
731,0,0
732,1,1
733,0,0


Se utiliza la metrica accuracy de sklearn para conocer la precisión del modelo

In [26]:
acc = accuracy_score(y_valid , y_preds)

In [27]:
acc

0.6639455782312925

Se guarda el mejor modelo para su uso posterior con datos nuevos

In [28]:
import joblib
# Guardar el modelo en un archivo
joblib.dump(best_xgb, 'mejor_xgb.pkl')

['mejor_xgb.pkl']